In [91]:
import os
import sys
import math
import random
import numpy as np
import pandas as pd
from pathlib import Path

import config as cfg
from helper import *

import data_reader
import data_writer

def create_inters(var_dict, streets):
    inter_outgoings = {inter_id: [] for inter_id in range(var_dict["num_inters"])}
    inter_incomings = {inter_id: [] for inter_id in range(var_dict["num_inters"])}
    for street in streets:
        inter_outgoings[street["start_inter"]].append(street)
        inter_incomings[street["end_inter"]].append(street)
    return inter_outgoings, inter_incomings


def stat_street_occurs(var_dict, streets, cars):
    street_occurs = {street["street_name"]: 0 for street in streets}

    for car in cars:
        for street_name in car["streets"]:
            street_occurs[street_name] += 1

    return street_occurs


def compute_street_values(streets, street_occurs, street_costs):
    street_values = {}
    for street in streets:
        occur = street_occurs[street["street_name"]]
        cost = street_costs[street["street_name"]]
        value = occur / cost
        street_values[street["street_name"]] = value
    return street_values


def solve(var_dict, streets, cars):
    inter_outgoings, inter_incomings = create_inters(var_dict, streets)

    # compute some features of the streets, car, ...
    street_occurs = stat_street_occurs(var_dict, streets, cars)
    street_costs = {street["street_name"]: street["cost"] for street in streets}
    # compute synthesised street values
    street_values = compute_street_values(streets, street_occurs, street_costs)

    # print some values, just to see
    street_values_sorted = sorted(list(street_values.items()), key=lambda e: e[1], reverse=True)
    print(street_values_sorted[:10])

    schedules = []
    for inter_id in range(var_dict["num_inters"]):
        street_outgoings = inter_outgoings[inter_id]
        street_incomings = inter_incomings[inter_id]

        green_lights = []

        subset_occurs = [street_occurs[street["street_name"]] for street in street_incomings]
        total_street_occurs = sum(subset_occurs)
        # we apply weighted street strategy only when the incoming streets are occuring in the car routes
        # if not, we can just set the duration of every incoming street to 1, they don't affect any results
        if total_street_occurs > 0:
            # compute the street values of all incoming streets
            incoming_values = [street_values[street["street_name"]] for street in street_incomings]
            incoming_values = [x for x in incoming_values if x != 0]
            incoming_streets = len(incoming_values)
            min_value = min(incoming_values)
            max_value = max(incoming_values)
            for street in street_incomings:
                if incoming_streets < 3:
                    duration = max(math.floor((street_values[street["street_name"]] - min_value) / max_value * 5), 1)
                else:
                    duration = max(math.floor((street_values[street["street_name"]] - min_value) / max_value * 2), 1)
                if duration <= 0:
                    continue
                green_lights.append((street["street_name"], duration))
        else:
            # but why there are intersections without any cars passing?
            for street in street_incomings:
                green_lights.append((street["street_name"], 1))

        num_incoming_streets = len(green_lights)
        schedules.append((inter_id, num_incoming_streets, green_lights))

    return schedules


In [92]:

def run(dset):
    input_path = f"{cfg.INPUT_DIR}/{dset}"
    output_path = f"{cfg.OUTPUT_DIR}_vincent/{dset}"

    var_dict, streets, cars = data_reader.read_input(input_path)
    schedules = solve(var_dict, streets, cars)
    #score = evaluate.estimate_score(output_data)
    data_writer.write_output(output_path, schedules)

    #print(f"Solution saved to {output_path}, score: {score}")


def run_all():
    for dset in cfg.DATASET_ALL:
        print(f"solving {dset}...")
        run(dset)

run_all()


solving a.txt...
[('rue-de-londres', 2.0), ('rue-d-amsterdam', 1.0), ('rue-d-athenes', 1.0), ('rue-de-moscou', 0.6666666666666666), ('rue-de-rome', 0.5)]
solving b.txt...
[('cfbf-ebia', 0.8695652173913043), ('debe-debb', 0.8695652173913043), ('ebia-ebgf', 0.8695652173913043), ('cfbg-cfbb', 0.7916666666666666), ('dhij-dhja', 0.7916666666666666), ('ebib-cfbc', 0.782608695652174), ('ebgf-ebgg', 0.7692307692307693), ('ddde-dddh', 0.7368421052631579), ('fcbh-fcbg', 0.7222222222222222), ('cfbc-cfbe', 0.6923076923076923)]
solving c.txt...
[('fife-fhgi', 47.0), ('fhha-fhhb', 47.0), ('dih-chh', 45.0), ('fdgc-fdgb', 42.0), ('fgfa-ffhi', 39.0), ('beeh-bfbj', 39.0), ('fedg-fedh', 38.0), ('bdc-cag', 38.0), ('ehg-ehh', 36.0), ('cag-caf', 36.0)]
solving d.txt...
[('bbef-cagi', 9.0), ('bhef-bdjc', 9.0), ('dfij-hd', 8.0), ('bcce-jgj', 7.0), ('cgee-bcdj', 7.0), ('hfaa-cdaf', 7.0), ('feic-eaaf', 7.0), ('ggih-efbc', 7.0), ('j-fabc', 6.0), ('dhhb-jb', 6.0)]
solving e.txt...
[('dai-daj', 11.0), ('ehe-ehf', 

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('/home/jupyter/google-hashcode-2021/src')

from pathlib import Path
DIR = Path('/home/jupyter/google-hashcode-2021')
input_dir = DIR / 'inputs'
output_dir = DIR / 'outputs'

In [56]:
from data_reader import read_input

info, streets, cars = read_input(input_dir / 'a.txt')
info, streets, cars

({'duration': 6,
  'num_inters': 4,
  'num_streets': 5,
  'num_cars': 2,
  'bonus': 1000},
 [{'start_inter': 2,
   'end_inter': 0,
   'street_name': 'rue-de-londres',
   'cost': 1},
  {'start_inter': 0,
   'end_inter': 1,
   'street_name': 'rue-d-amsterdam',
   'cost': 1},
  {'start_inter': 3,
   'end_inter': 1,
   'street_name': 'rue-d-athenes',
   'cost': 1},
  {'start_inter': 2, 'end_inter': 3, 'street_name': 'rue-de-rome', 'cost': 2},
  {'start_inter': 1,
   'end_inter': 2,
   'street_name': 'rue-de-moscou',
   'cost': 3}],
 [{'num_streets': 4,
   'streets': ['rue-de-londres',
    'rue-d-amsterdam',
    'rue-de-moscou',
    'rue-de-rome']},
  {'num_streets': 3,
   'streets': ['rue-d-athenes', 'rue-de-moscou', 'rue-de-londres']}])

In [54]:
def get_street_map(info, streets):
    street_map = dict(enumerate([street['street_name'] for street in streets], info['num_inters']))
    street_map = {v: k for k, v in street_map.items()}
    
    return street_map

def get_cost_dict(streets, street_map, from_to_map):
    cost_dict = {(k, v): v for k, v in from_to_map.items()}
    street_cost_dict = {street_map[street['street_name']]: street['cost'] for street in streets}
    final_cost_dict = {}
    for k, v in cost_dict.items():
        if v in street_cost_dict:
            final_cost_dict[k] = street_cost_dict[v]
        else:
            final_cost_dict[k] = -1
    
    return final_cost_dict

def get_from_to_map(streets, street_map):
    from_to_map = {street['start_inter']: street_map[street['street_name']] for street in streets}
    print(from_to_map)
    from_to_map.update({street_map[street['street_name']]: street['end_inter'] for street in streets})
    print(from_to_map)
    return from_to_map

def get_routes(cars, street_map):
    routes = []
    for car in cars:
        route = []
        for street in car['streets']:
            route.append(street_map[street])

        routes.append(route)
    print(routes)
    return routes

def init_states(info, routes, from_to_map):
    states = {(k, v): [] for k, v in from_to_map.items()}
    
    for route in routes:
        node = from_to_map[route[0]]
        state = states.get((route[0], node))
        state.append((0, route[1:]))
        states[node] = state
    
    return states

In [55]:
def run(info, streets, cars):
    
    car_bonus = info['bonus']
    
    street_map = get_street_map(info, streets)
    from_to_map = get_from_to_map(streets, street_map)
    cost_dict = get_cost_dict(streets, street_map, from_to_map)
    print(cost_dict)
      
    routes = get_routes(cars, street_map)
    intermediate_score = 0
    for time in range(info['duration']):
        if time == 0:
            states = init_states(info, routes, from_to_map)
        print(states)
        state_updates = []
        for node, cars in states.items():
            if cars == []:
                continue
            node_cost = cost_dict[node]
            
            # Red light
            if node_cost == -1:
                next_step = True
                #continue
            # Green light
            elif node_cost == 0:
                next_step = True
            # On the road
            elif node_cost > 0:
                start_time, _ = cars[0]    
                time_elapsed = time - start_time
                if node_cost == time_elapsed:
                    next_step = True
                else:
                    continue

            if next_step:
                car = cars.pop(0)
                route_left = car[1]
                
                if len(route_left) == 0:
                    intermediate_score += info['duration'] - time
                    continue
                
                new_node = route_left[0]
                new_car = (time, route_left[1:])
                
                state_updates.append([(node[1], new_node), new_car])
                
        for new_node, new_car in state_updates:
            state = states.get(new_node)
            if state is None:
                states[new_node] = [new_car]
            else:
                states[new_node].append(new_car)
            
        
    cars_left = sum([len(cars) for node, cars in states.items()])
    score = (info['num_cars'] - cars_left)*car_bonus + intermediate_score
    
    return score

    
run(info, streets, cars)

{2: 7, 0: 5, 3: 6, 1: 8}
{2: 7, 0: 5, 3: 6, 1: 8, 4: 0, 5: 1, 6: 1, 7: 3, 8: 2}
{(2, 7): 2, (0, 5): 1, (3, 6): 1, (1, 8): 3, (4, 0): -1, (5, 1): -1, (6, 1): -1, (7, 3): -1, (8, 2): -1}
[[4, 5, 8, 7], [6, 8, 4]]
{(2, 7): [], (0, 5): [], (3, 6): [], (1, 8): [], (4, 0): [(0, [5, 8, 7])], (5, 1): [], (6, 1): [(0, [8, 4])], (7, 3): [], (8, 2): [], 0: [(0, [5, 8, 7])], 1: [(0, [8, 4])]}
{(2, 7): [], (0, 5): [(0, [8, 7])], (3, 6): [], (1, 8): [(0, [4])], (4, 0): [], (5, 1): [], (6, 1): [], (7, 3): [], (8, 2): [], 0: [], 1: []}
{(2, 7): [], (0, 5): [], (3, 6): [], (1, 8): [(0, [4])], (4, 0): [], (5, 1): [], (6, 1): [], (7, 3): [], (8, 2): [], 0: [], 1: [], (5, 8): [(1, [7])]}


KeyError: (5, 8)